In [55]:
%load_ext rpy2.ipython

In [56]:
%%R
library(lmtest)

R[write to console]: Loading required package: zoo

R[write to console]: 
Attaching package: ‘zoo’


R[write to console]: The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




In [57]:
%%R
library(lme4)
library(lmerTest)

R[write to console]: Loading required package: Matrix

R[write to console]: 
Attaching package: ‘lmerTest’


R[write to console]: The following object is masked from ‘package:lme4’:

    lmer


R[write to console]: The following object is masked from ‘package:stats’:

    step




In [ ]:
import random

import numpy as np
import pandas as pd
import seaborn as sns

from construal_shifting.task_modeling.model_fitter import ModelFitter
from construal_shifting.task_modeling.participant_model import ParticipantModel

from data_analysis import download_data, download_condition_counts, calc_condition_counts, ExperimentDataLoader


In [2]:
exp_data = ExperimentDataLoader(
    trialdata_file="rawtrialdata-anon.csv"
)
summary_df = pd.read_json('./data/summary_df.json')
all_participant_data = [p for p in exp_data.completed_participant_data() if p.sessionId in summary_df.sessionId.values]

## Did gain from taking a notch make it more likely to take it?

In [41]:
from tqdm import tqdm
from msdm.algorithms import PolicyIteration
from construal_shifting.fastgridworld import GridWorld2 as GridWorld

gw_params = dict(
    step_cost=-1,
    discount_rate=1,
)

notch_gain_stats = []

for p in tqdm(all_participant_data):
    for t in p.test_trials():
        notch_gw = GridWorld(**{
            **t.invtransformed_GridWorld_params,
            **gw_params,
            'wall_features': "#ABCDEFG",
        })
        block_gw = GridWorld(**{
            **t.invtransformed_GridWorld_params,
            **gw_params,
            'wall_features': "#ABCDEFGabcdefg",
        })
        notch_plan = PolicyIteration().plan_on(notch_gw)
        block_plan = PolicyIteration().plan_on(block_gw)
        notch_gain = notch_plan.initial_value - block_plan.initial_value
        notch_gain_stats.append({
            'sessionId': p.sessionId,
            'trial_index': t.trial_index,
            'notch_gain': notch_gain,
            'visited_critical_notch': t.visited_critical_notch,
            'condition_name': p.condition_name,
        })

  0%|          | 0/377 [00:00<?, ?it/s]/Users/markho/Dropbox/Cog-Sci-Research/Projects-active/construal-shifting-public/env37/lib/python3.7/site-packages/msdm/algorithms/policyiteration.py:41: UserWarning: MDP contains states that never reach an absorbing state. Values for these states will be set using self.undefined_value=0
  f"Values for these states will be set using self.undefined_value={self.undefined_value}"
100%|██████████| 377/377 [01:02<00:00,  6.00it/s]


In [78]:
import pandas as pd
df = pd.DataFrame(notch_gain_stats)
first_test_df = df[df.trial_index == 22]

In [80]:
pd.crosstab(first_test_df['visited_critical_notch'], first_test_df['condition_name'], first_test_df['notch_gain'], aggfunc='mean')

condition_name,cccccccccccc,ffffffffffff
visited_critical_notch,,
False,7.123810,6.857143
True,6.915663,6.981366


In [81]:
%%R -i first_test_df
first_test_df$condition_name <- factor(
    first_test_df$condition_name,
    levels=c('ffffffffffff', 'cccccccccccc')
)
contrasts(first_test_df$condition_name) <- contr.sum(2)*.5
print(contrasts(first_test_df$condition_name))
base_mod <- glm(
    visited_critical_notch ~ 1,
    data=first_test_df,
    family=binomial
)
model_comp_crit_notch <- anova(
    base_mod,
    update(base_mod, . ~ + notch_gain),
    test='Chisq'
)
print(model_comp_crit_notch)

             [,1]
ffffffffffff  0.5
cccccccccccc -0.5
Analysis of Deviance Table

Model 1: visited_critical_notch ~ 1
Model 2: visited_critical_notch ~ notch_gain
  Resid. Df Resid. Dev Df Deviance Pr(>Chi)
1       376     489.46                     
2       375     488.45  1   1.0168   0.3133
